# Tutorial 06, case 7b: Stokes problem with Neumann control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Gamma_{obs}} (v - v_d)^2 dx + \frac{\alpha_1}{2} \int_{\Gamma_C} |\nabla_{\mathbf{t}} u|^2 ds + \frac{\alpha_2}{2} \int_{\Gamma_C} |u|^2 ds$$
s.t.
$$\begin{cases}
- \nu \Delta v + \nabla p = f       & \text{in } \Omega\\
             \text{div} v = 0       & \text{in } \Omega\\
                        v = g       & \text{on } \Gamma_{in}\\
                        v = 0       & \text{on } \Gamma_{w}\\
   p n - \nu \partial_n v = u       & \text{on } \Gamma_{C}
\end{cases}$$

where
$$\begin{align*}
& \Omega                      & \text{unit square}\\
& \Gamma_{in}                 & \text{has boundary id 1}\\
& \Gamma_{w}                  & \text{has boundary id 2}\\
& \Gamma_{C}                  & \text{has boundary id 3}\\
& \Gamma_{obs}                & \text{has boundary id 4}\\
& u \in [L^2(\Gamma_C)]^2     & \text{control variable}\\
& v \in [H^1(\Omega)]^2       & \text{state velocity variable}\\
& p \in L^2(\Omega)           & \text{state pressure variable}\\
& \alpha_1, \alpha_2 > 0      & \text{penalization parameters}\\
& v_d                         & \text{desired state}\\
& f                           & \text{forcing term}\\
& g                           & \text{inlet profile}\\
\end{align*}$$
using an adjoint formulation solved by a one shot approach

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import (as_vector, div, FacetNormal, grad, inner, Measure, replace, SpatialCoordinate,
                 TestFunction, TrialFunction)
from dolfinx import Constant, DirichletBC, Function, FunctionSpace, VectorFunctionSpace
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plot import create_vtk_topology
from multiphenicsx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                               BlockVecSubVectorWrapper, create_vector_block, DofMapRestriction)
import pyvista

### Mesh

In [ ]:
if MPI.COMM_WORLD.size > 1:
    mesh_ghost_mode = GhostMode.shared_facet  # shared_facet ghost mode is required by dS
else:
    mesh_ghost_mode = GhostMode.none
with XDMFFile(MPI.COMM_WORLD, "data/bifurcation.xdmf", "r") as infile:
    mesh = infile.read_mesh(mesh_ghost_mode)
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_1 = boundaries.indices[boundaries.values == 1]
boundaries_2 = boundaries.indices[boundaries.values == 2]
boundaries_3 = boundaries.indices[boundaries.values == 3]
boundaries_12 = boundaries.indices[np.isin(boundaries.values, (1, 2))]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)
dS = Measure("dS")(subdomain_data=boundaries)

In [ ]:
# Normal and tangent
n = FacetNormal(mesh)
t = as_vector([n[1], -n[0]])

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

### Function spaces

In [ ]:
Y_velocity = VectorFunctionSpace(mesh, ("Lagrange", 2))
Y_pressure = FunctionSpace(mesh, ("Lagrange", 1))
U = VectorFunctionSpace(mesh, ("Lagrange", 2))
Q_velocity = Y_velocity.clone()
Q_pressure = Y_pressure.clone()

### Restrictions

In [ ]:
dofs_Y_velocity = np.arange(0, Y_velocity.dofmap.index_map.size_local + Y_velocity.dofmap.index_map.num_ghosts)
dofs_Y_pressure = np.arange(0, Y_pressure.dofmap.index_map.size_local + Y_pressure.dofmap.index_map.num_ghosts)
dofs_U = locate_dofs_topological(U, boundaries.dim, boundaries_3)
dofs_Q_velocity = dofs_Y_velocity
dofs_Q_pressure = dofs_Y_pressure
restriction_Y_velocity = DofMapRestriction(Y_velocity.dofmap, dofs_Y_velocity)
restriction_Y_pressure = DofMapRestriction(Y_pressure.dofmap, dofs_Y_pressure)
restriction_U = DofMapRestriction(U.dofmap, dofs_U)
restriction_Q_velocity = DofMapRestriction(Q_velocity.dofmap, dofs_Q_velocity)
restriction_Q_pressure = DofMapRestriction(Q_pressure.dofmap, dofs_Q_pressure)
restriction = [restriction_Y_velocity, restriction_Y_pressure, restriction_U,
               restriction_Q_velocity, restriction_Q_pressure]

### Trial and test functions

In [ ]:
(v, p) = (TrialFunction(Y_velocity), TrialFunction(Y_pressure))
(w, q) = (TestFunction(Y_velocity), TestFunction(Y_pressure))
u = TrialFunction(U)
r = TestFunction(U)
(z, b) = (TrialFunction(Q_velocity), TrialFunction(Q_pressure))
(s, d) = (TestFunction(Q_velocity), TestFunction(Q_pressure))

 ### Problem data

In [ ]:
nu = 0.04
alpha_1 = 0.001
alpha_2 = 0.1 * alpha_1
x = SpatialCoordinate(mesh)
a = 1.0
c = 0.8
v_d = as_vector((a * (c * 10.0 * (x[1]**3 - x[1]**2 - x[1] + 1.0))
                 + ((1.0 - c) * 10.0 * (-x[1]**3 - x[1]**2 + x[1] + 1.0)), 0.0))
ff = Constant(mesh, (0., 0.))


def g_eval(x):
    values = np.zeros((2, x.shape[1]))
    values[0, :] = 10.0 * a * (x[1, :] + 1.0) * (1.0 - x[1, :])
    return values


g = Function(Y_velocity)
g.interpolate(g_eval)
bc0 = Function(Y_velocity)

### Optimality conditions

In [ ]:
def tracking(v, w):
    return inner(v, w)("-")


def penalty(u, r):
    return alpha_1 * inner(grad(u) * t, grad(r) * t) + alpha_2 * inner(u, r)


a = [[tracking(v, w) * dS(4), None, None, nu * inner(grad(z), grad(w)) * dx, - b * div(w) * dx],
     [None, None, None, - q * div(z) * dx, None],
     [None, None, penalty(u, r) * ds(3), - inner(z, r) * ds(3), None],
     [nu * inner(grad(v), grad(s)) * dx, - p * div(s) * dx, - inner(u, s) * ds(3), None, None],
     [- d * div(v) * dx, None, None, None, None]]
f = [tracking(v_d, w) * dS(4),
     None,
     None,
     inner(ff, s) * dx,
     None]
a[0][0] += Constant(mesh, 0.) * inner(v, w) * (ds(1) + ds(2))
a[3][3] = Constant(mesh, 0.) * inner(z, s) * (ds(1) + ds(2))
f[1] = Constant(mesh, 0.) * q * dx
f[2] = inner(Constant(mesh, (0., 0.)), r) * dx
f[4] = Constant(mesh, 0.) * d * dx
bdofs_Y_velocity_1 = locate_dofs_topological((Y_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_1)
bdofs_Y_velocity_2 = locate_dofs_topological((Y_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_2)
bdofs_Q_velocity_12 = locate_dofs_topological((Q_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_12)
bc = [DirichletBC(g, bdofs_Y_velocity_1, Y_velocity), DirichletBC(bc0, bdofs_Y_velocity_2, Y_velocity),
      DirichletBC(bc0, bdofs_Q_velocity_12, Q_velocity)]

### Solution

In [ ]:
(v, p) = (Function(Y_velocity), Function(Y_pressure))
u = Function(U)
(z, b) = (Function(Q_velocity), Function(Q_pressure))

### Cost functional

In [ ]:
J = 0.5 * tracking(v - v_d, v - v_d) * dS(4) + 0.5 * penalty(u, u) * ds(3)

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = [[replace(a[i][j], {s: w, d: q}) if a[i][j] is not None else None
            for j in (0, 1)] for i in (3, 4)]
f_state = [replace(f[i], {s: w, d: q}) for i in (3, 4)]
bc_state = [bc[0], bc[1]]

In [ ]:
# Assemble the block linear system for the state
A_state = assemble_matrix_block(a_state, bcs=bc_state,
                                restriction=([restriction[i] for i in (3, 4)],
                                             [restriction[j] for j in (0, 1)]))
A_state.assemble()
F_state = assemble_vector_block(f_state, a_state, bcs=bc_state,
                                restriction=[restriction[i] for i in (3, 4)])

In [ ]:
# Solve
vp = create_vector_block([f[j] for j in (0, 1)], restriction=[restriction[j] for j in (0, 1)])
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, vp)
vp.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(vp, [c.function_space.dofmap for c in (v, p)]) as vp_wrapper:
    for vp_wrapper_local, component in zip(vp_wrapper, (v, p)):
        with component.vector.localForm() as component_local:
            component_local[:] = vp_wrapper_local

In [ ]:
J_uncontrolled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 2.8479865)

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
def pyvista_vector_field_plot(mesh, vector_field, name, factor):
    grid = dolfinx_to_pyvista_mesh(mesh)
    values = np.zeros((mesh.geometry.x.shape[0], 3))
    values[:, :2] = vector_field.compute_point_values()
    grid.point_arrays[name] = values
    grid.set_active_vectors(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    glyphs = grid.glyph(orient=name, factor=factor)
    plotter.add_mesh(glyphs)
    plotter.show()

In [ ]:
pyvista_vector_field_plot(mesh, v, "uncontrolled state velocity", factor=1e-2)

In [ ]:
pyvista_scalar_field_plot(mesh, p, "uncontrolled state pressure")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
A = assemble_matrix_block(a, bcs=bc, restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=bc, restriction=restriction)

In [ ]:
# Solve
vpuzb = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, vpuzb)
vpuzb.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
with BlockVecSubVectorWrapper(vpuzb, [c.function_space.dofmap for c in (v, p, u, z, b)],
                              restriction) as vpuzb_wrapper:
    for vpuzb_wrapper_local, component in zip(vpuzb_wrapper, (v, p, u, z, b)):
        with component.vector.localForm() as component_local:
            component_local[:] = vpuzb_wrapper_local

In [ ]:
J_controlled = mesh.mpi_comm().allreduce(assemble_scalar(J), op=MPI.SUM)
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 1.7643950)

In [ ]:
pyvista_vector_field_plot(mesh, v, "state velocity", factor=1e-2)

In [ ]:
pyvista_scalar_field_plot(mesh, p, "state pressure")

In [ ]:
pyvista_vector_field_plot(mesh, u, "control", factor=1e-1)

In [ ]:
pyvista_vector_field_plot(mesh, z, "adjoint velocity", factor=1e-1)

In [ ]:
pyvista_scalar_field_plot(mesh, b, "adjoint pressure")